**This Week’s Fiddler**
A teacher is handing out candy to his students, of which there are at least four. He abides by the following rules:
- He hands out candy to groups of three students (i.e., “trios”) at a time. Each member of the trio gets one piece of candy.
- Each unique trio can ask for candy, but that same trio can’t come back for seconds. If students in the trio want more candy, they must return as part of a different trio.
- When a trio gets candy, the next trio can’t contain any students from that previous trio.
It turns out that every possible trio can get a helping of candy. What is the smallest class size for which this is possible?

**This Week’s Extra Credit**
Instead of trios of students, suppose now that groups of 10 students come up to get candy. This time, there are at least 11 students in the class. As before:
- Each member of the group of 10 gets one piece of candy per visit.
- Each unique group of 10 can ask for candy, but the exact same group of 10 can’t come back for seconds. If students in the group want more candy, they must return as part of a different group.
- When a group of 10 gets candy, the next group of 10 can’t contain any students from the previous group of 10.

Suppose the class size is the minimum that allows every possible group of 10 to get a helping of candy. How many pieces of candy does each student receive?

---

Let T be the group size (3 or 10 above).
We need at least 2T + 1 students. because with 2T, we'll be stuck after 2 groups.
The hard part seems to be showing that 2T+1 is sufficient.

For group size T = 2, we can show that a class size C = 5 = 2*T+1 suffices by hand, if groups go the teacher in the following order. 
("AB" means the group with students A and B)
   AB CD EA BC DE     AC BD CE DA EB

Same thing written using numbers:
   12 34 51 23 45     13 24 35 41 52

-----

For T = 3, C=7, there are 7.6.5./1.2.3 = 35 groups / trios.
Thinking of 1 .. 7 as being on a circle, we can list the groups as follows.

##### ABC - Successive
123 456 712 345 671 234 567

##### ACE - Alternate
135 246 357 461 572 613 724

##### ACD - Skip 1 first.
134 245 356 467 571 612 723

##### ABE - Skip 2.
125 236 347 451 562 673 714

##### ABD - Skip 1 later.
124 235 346 457 561 672 713

The first 2 patterns of groups (successive, alternate) mesh nicely into tidy loops, in the order above.
The other 3 don't.
But for the other 3, notice that we can interleave the three groups (ACD  EFB(ABE)  GAC(ABD)).
So, those 3 groups can be combined into one larger loop as follows.

##### Interleaved
134 562 713
245 673 124
356 714 235
467 125 346
571 236 457
612 347 561
723 451 672

Let's try to put it all together.
Looks like we can strt with the Successive loop, follow with the Interleaved loop, and end with the  Alternate loop, satisfying the constraints where the loops join.

123 456 712 345 671 234 567 
-> 
134 562 713
245 673 124
356 714 235
467 125 346
571 236 457
612 347 561
723 451 672
->
135 246 357 461 572 613 724

TA-DA.

**I have convinced, at least myself, that a class size of 7 is sufficient for groups of size 3.**

Also, phew. 
Clearly, groups of size 10 cannot be done by hand. :)

---



So, let's try making up some code to make up the graph of groups, and check for the existence of some hamiltonian path, given the allowed rules (expressed as edges). Our graph data structure will be a dict with all the nodes as keys, and the list of adjacent vertices for each node as the value.

In [24]:
import itertools
names = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

def make_graph(class_size, group_size):
    assert(class_size <= 54) # because we only listed 54 names above.
    students = names[0:class_size]
    set_students = set(students)
    graph = {}
    for group_t in itertools.combinations(students,group_size):
        group = "".join(sorted(group_t))
        graph[group] = []
        group_complement = set_students - set(group)
        for neighbor_t in itertools.combinations(group_complement, group_size):
            neighbor = "".join(sorted(neighbor_t))
            graph[group].append(neighbor)
    return graph

make_graph(5,2)
    


{'AB': ['CD', 'CE', 'DE'],
 'AC': ['BD', 'BE', 'DE'],
 'AD': ['BC', 'CE', 'BE'],
 'AE': ['BC', 'CD', 'BD'],
 'BC': ['AD', 'DE', 'AE'],
 'BD': ['AC', 'CE', 'AE'],
 'BE': ['AD', 'CD', 'AC'],
 'CD': ['AB', 'BE', 'AE'],
 'CE': ['BD', 'AD', 'AB'],
 'DE': ['BC', 'AC', 'AB']}

In [29]:
def find_hamiltonian_path(graph, start_node):
    num_nodes = len(graph.keys())
    neighbors_per_node = len(graph[start_node])
    #print(num_nodes, neighbors_per_node, start_node)
    path = [start_node]
    idxs = {}
    while (len(path) < num_nodes):
        last_node = path[-1]
        #print(last_node)
        if (last_node not in idxs):
            idxs[last_node] = 0
        while(True):
            idx = idxs[last_node]
            idxs[last_node] += 1
            #print(idx)
            if (idx >= neighbors_per_node):
                path.pop()
                break
            new_node = graph[last_node][idx]
            if (new_node not in path):
                path.append(new_node)
                idxs[new_node] = 0
                break                             
        if (idxs[start_node] >= neighbors_per_node):
            break
    return path

In [ ]:
def make_and_check(group_size):
    class_size = 2 * group_size + 1
    G = make_graph(class_size, group_size)
    print("graph created")
    start_node = names[0:group_size]
    path = find_hamiltonian_path(G, start_node)
    if (len(path) <= 1):
        print("No path found")
    else :
        print("Found path of length", len(path), ":")
    print(path)

make_and_check(2)

Found path of length 10 :
['AB', 'CD', 'BE', 'AD', 'CE', 'BD', 'AC', 'DE', 'BC', 'AE']


With this, we can solve the base fiddler again.

In [35]:
make_and_check(3)

Found path of length 35 :
['ABC', 'DFG', 'BCE', 'ADF', 'BCG', 'ADE', 'BFG', 'ACD', 'BEG', 'CDF', 'ABG', 'DEF', 'ACG', 'BEF', 'ADG', 'CEF', 'ABD', 'CFG', 'ABE', 'CDG', 'AEF', 'BDG', 'ACE', 'BDF', 'CEG', 'ABF', 'CDE', 'AFG', 'BDE', 'ACF', 'DEG', 'BCF', 'AEG', 'BCD', 'EFG']


Looks good.

----

Let's work our way up towards 10.

In [ ]:
make_and_check(4)

Um, this is not looking good. Groups of size 3 took 0.2 seconds. Groups of size 4 took 10 minutes and was still running when I stopped it. So, clearly my homebrewed algorithm is not going to cut it for group of size 10.

---

Seems like this problem is NP-hard. So, this approach of writing or getting some code to solve a 300k vertex graph is unlikely to work.

----



That said, given that the graph is symmetric, and for group size 10 and class size 21, each node will have 11 neighbors, so I _think_ it is extremely likely that the graph will have an hamiltonian path.

With that assumed, it's easy to count the number of pieces of candy.

To make a group of 10 involving a given student (A), we need to pick 9 other students.
This can be done 20 choose 9 ways = 
= 20! / 9! 11! = 167960 ways.
So, student A will get 167960 pieces of candy, one per group that A is part of.

**By symmetry, all students will get 167960 pieces of candy.**